<a href="https://colab.research.google.com/github/faisalisafk/python_scripts/blob/main/Named_entity_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd
import spacy
from wordcloud import WordCloud, STOPWORDS
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import re
import random
import math
import copy
df = pd.read_csv('data.csv')

In [5]:
df.sample(10)

,transaction_descriptor,store_number,dataset
1,AUTOZONE #3547,3547,train
19,EXPRESS#0920,920,train
109,HOLIDAY STNSTORE 0453,453,validation
113,NNT FAMOUSFOOTWEAR#570903,570903,validation
241,NST BEST BUY #59 060251,59,test
135,NNT HIBBETT SPORTS 072473,72473,validation
79,BIG O TIRES #6255 GRAND JUNCTIOCO,6255,train
261,NST BEST BUY #188 871025,188,test
235,NST BEST BUY #401 731468,401,test
175,TACO BELL #029071,29071,validation


In [19]:
df.count()

transaction_descriptor    300
store_number              300
dataset                   300
dtype: int64

In [6]:
nlp0 = spacy.load('en_core_web_sm')

In [7]:
nlp0.pipe_names

['tagger', 'parser', 'ner']

In [8]:
ner0 = nlp0.get_pipe('ner')

In [10]:
ner0.add_label

<function EntityRecognizer.add_label>

In [11]:
def process_review(review):
    processed_token = []
    for token in review.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    return ' '.join(processed_token)

In [12]:
all_drugs = df['store_number'].unique().tolist()

In [13]:
all_drugs = [x.lower() for x in all_drugs]

In [14]:
all_drugs

['2257',
 '3547',
 '1485',
 '3',
 '568',
 '40',
 '1847',
 '2650',
 '535',
 '1305',
 '2202',
 '26824',
 '688',
 '4681',
 '227',
 '3207',
 '1040',
 'f1013',
 '207812',
 '920',
 'f16829',
 '208998',
 '49134',
 '302900',
 '1251',
 '479',
 '354',
 '7430',
 '483280353',
 'f5579',
 '10262660',
 '447',
 '25',
 '196',
 '516260',
 '6870',
 '45518',
 'f2128',
 '22357',
 '748361300',
 'f3172',
 '9505',
 '1407',
 '1548',
 '6524',
 '6425',
 '6853',
 '308296',
 '5546',
 '142',
 '11572299',
 '3857',
 '1327',
 '27',
 '303',
 '24',
 '209',
 '7350',
 '1783',
 '1316',
 '4102',
 '11002986',
 '98',
 '1774',
 '2910',
 '2315',
 '10630845',
 '42',
 '437',
 '572264',
 '171793',
 '3312162',
 '4',
 '1858',
 'f26490',
 '974',
 '881706',
 '3626',
 '6286',
 '6255',
 '14560807',
 '739723',
 '113',
 '14782798',
 '25139',
 '9386',
 '12862950',
 '7591',
 '2',
 '500',
 'f35869',
 '470811',
 '374572602',
 '9502',
 'g020029',
 '355514',
 '6828',
 '1129',
 '34',
 '485',
 '833',
 '472605',
 '5320',
 '337734',
 'f565',
 '1382

In [15]:
df['transaction_descriptor']

0      DOLRTREE 2257 00022574 ROSWELL
1                      AUTOZONE #3547
2               TGI FRIDAYS 1485 0000
3              BUFFALO WILD WINGS 003
4                      J. CREW #568 0
                    ...              
295                  MCDONALD'S F2151
296         NST BEST BUY #1403 332411
297               CVS/PHARMACY #06689
298             BANANA REPUBLIC #8109
299                BOSTON MARKET 0443
Name: transaction_descriptor, Length: 300, dtype: object

In [61]:
count = 0
TRAIN_DATA = []
for _, item in df.iterrows():
    ent_dict = {}
    if count < 1000:
        review = process_review(item['transaction_descriptor'])
        #Locate drugs and their positions once and add to the visited items.
        visited_items = []
        entities = []
        for token in review.split():
            if token in all_drugs:
                for i in re.finditer(token, review):
                    if token not in visited_items:
                        entity = (i.span()[0], i.span()[1], 'store_number')
                        visited_items.append(token)
                        entities.append(entity)
        if len(entities) > 0:
            ent_dict['entities'] = entities
            train_item = (review, ent_dict)
            TRAIN_DATA.append(train_item)
            count+=1

In [62]:
len(TRAIN_DATA)

203

In [85]:
n_iter = 50

def train_ner(training_data):
    """Steps
    Create a Blank NLP  model object
    Create and add NER to the NLP model
    Add Labels from your training data
    Train  
    """
    best = math.inf
    TRAIN_DATA = training_data
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.6,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)
        if losses['ner'] < best:
            best = losses['ner']
            best_nlp = copy.deepcopy(nlp)
            print('best changed')
    return best_nlp

In [86]:
nlp2 = train_ner(TRAIN_DATA)

Created blank 'en' model
Losses {'ner': 317.5577306523919}
best changed
Losses {'ner': 185.90634988219244}
best changed
Losses {'ner': 95.52256159144773}
best changed
Losses {'ner': 24.06420129384459}
best changed
Losses {'ner': 12.140726769382184}
best changed
Losses {'ner': 9.814573905356587}
best changed
Losses {'ner': 19.683606193839466}
Losses {'ner': 7.590761242858728}
best changed
Losses {'ner': 9.917655509097253}
Losses {'ner': 2.0002250641183137}
best changed
Losses {'ner': 7.392037750692789}
Losses {'ner': 2.000781352935874}
Losses {'ner': 3.253938042853766}
Losses {'ner': 6.1089011529043304}
Losses {'ner': 5.875557716887819}
Losses {'ner': 3.1995455190063957}
Losses {'ner': 4.479947566705671}
Losses {'ner': 3.24482265276773}
Losses {'ner': 2.0381122458667646}
Losses {'ner': 2.232219480939274}
Losses {'ner': 4.586786193603044}
Losses {'ner': 0.5693105612259409}
best changed
Losses {'ner': 2.0000354802883655}
Losses {'ner': 1.352973239696875}
Losses {'ner': 2.674155605539068}


In [98]:
temp = df.sample()
print(temp)
docx2 = nlp2(temp['transaction_descriptor'].values[0])

for entity in docx2.ents:
  print(entity)


      transaction_descriptor store_number     dataset
126  WENDY'S #1970 000001970         1970  validation
1970
